# Init for pyqcu.

In [1]:
import cupy as cp
import numpy as np
import functools
from scipy.sparse.linalg import LinearOperator, eigsh
import cupyx.scipy.sparse as cpx_sparse
from pyqcu import define
from pyqcu import io
from pyqcu import qcu
from pyqcu import eigen, cg, bistabcg
from opt_einsum import contract
from pyqcu.set import params, argv, set_ptrs
params[define._NODE_RANK_] = define.rank
params[define._NODE_SIZE_] = define.size
index = -1
params[define._LAT_X_] = 8
params[define._LAT_Y_] = 8
params[define._LAT_Z_] = 4
params[define._LAT_T_] = 8
n = int(params[define._LAT_X_] / define._LAT_P_ * params[define._LAT_Y_] *
        params[define._LAT_Z_] * params[define._LAT_T_] *
        define._LAT_S_*define._LAT_C_)
params[define._MG_X_] = 2
params[define._MG_Y_] = 2
params[define._MG_Z_] = 2
params[define._MG_T_] = 2
print(params)


    @@@@@@######QCU NOTES START######@@@@@@@
    0. Required: MPI(e.g. 4.1.2), CUDA(e.g. 12.4), CMAKE(e.g. 3.22.1), GCC(e.g. 11.4.0), HDF5-MPI(e.g. 1.10.7,'apt install libhdf5-mpi-dev && export HDF5_MPI="ON" && pip install --no-binary=h5py h5py').
    1. The libqcu.so was compiled when pyqcu setup in download_path/PyQCU/lib, please add this path to your LD_LIBRARY_PATH.
    2. The QCU(PyQCU) splite grid by x->y->z->t, lattice by x->y->z->t->p->d->c->c or x->y->z->t->c->s(->p) and x->y->z->t->c->s->c->s(->p).
    3. The QUDA(PyQUDA) splite grid by t->z->y->x, lattice by c->c->x->y->z->t->p->d or c->s->x->y->z->t(->p) and c->s->c->s->x->y->z->t(->p).
    4. The QCU input params in numpy array(dtype=np.int32), argv in  numpy array(dtype=np.float32 or float64) array, set_ptrs in numpy array(dtype=np.int64), other in cupy array(dtype=cp.complex64 or complex128).
    5. The smallest lattice size is (x=4,y=4,z=4,t=8) that QCU support.
    @@@@@@######QCU NOTES END######@@@@@@@
    
Parameter

# Give matvec.

In [2]:
def generate_sparse_complex_psd_matrix(n, density=0.1):
    real_part = cpx_sparse.random(
        n, n, density=density, format="csr", dtype=cp.float32)
    imag_part = cpx_sparse.random(
        n, n, density=density, format="csr", dtype=cp.float32)
    A = real_part + 1j * imag_part
    A_hermitian = A + A.getH()
    A_psd = A_hermitian + n * cpx_sparse.identity(n, dtype=cp.complex64)
    return A_psd


A = generate_sparse_complex_psd_matrix(n)
def matvec(src):
    return (A@src.flatten()).reshape(src.shape)


# Give guage's eigenvalues and eigenvectors to hdf5 files. (pass, don't run this)

In [41]:
eigenvalues, eigenvectors = eigen.solver(
    n=n, k=define._LAT_Ne_,matvec=matvec,dtype=A.dtype).run()
eigenvectors=io.eigenvectors2esctzyx(eigenvectors,params)

: 

# Run matvec(eigenvector[.]) ?= eigenvalue[.]*eigenvector[.] for eigen test. (pass, don't run this)

In [40]:
for i, ev in enumerate(eigenvalues):
    print(f"λ_{i} = {ev:.2e}")
    # Verify eigenvector
    v = eigenvectors[i]
    w = cp.zeros_like(v)
    w = matvec(v)
    error = cp.linalg.norm(w - ev * v) / cp.linalg.norm(w)
    print(f"Relative error: {error:.2e}")
    j = i+1
    if j == len(eigenvalues):
        j = 0
    print(
        f"Diff between λ_{i} and λ_{j}: {cp.linalg.norm(eigenvectors[i] - eigenvectors[j])/cp.linalg.norm(eigenvectors[i]):.2e}")

λ_0 = 1.35e+04+0.00e+00j
Relative error: 8.23e-06
Diff between λ_0 and λ_1: 1.41e+00
λ_1 = 1.24e+04+0.00e+00j
Relative error: 8.38e-05
Diff between λ_1 and λ_2: 1.41e+00
λ_2 = 1.24e+04+0.00e+00j
Relative error: 8.50e-05
Diff between λ_2 and λ_3: 1.41e+00
λ_3 = 1.24e+04+0.00e+00j
Relative error: 8.37e-05
Diff between λ_3 and λ_4: 1.41e+00
λ_4 = 1.24e+04+0.00e+00j
Relative error: 8.59e-05
Diff between λ_4 and λ_5: 1.41e+00
λ_5 = 1.24e+04+0.00e+00j
Relative error: 8.81e-05
Diff between λ_5 and λ_6: 1.41e+00
λ_6 = 1.24e+04+0.00e+00j
Relative error: 8.56e-05
Diff between λ_6 and λ_7: 1.41e+00
λ_7 = 1.24e+04+0.00e+00j
Relative error: 8.78e-05
Diff between λ_7 and λ_8: 1.41e+00
λ_8 = 1.24e+04+0.00e+00j
Relative error: 8.85e-05
Diff between λ_8 and λ_9: 1.41e+00
λ_9 = 1.24e+04+0.00e+00j
Relative error: 8.70e-05
Diff between λ_9 and λ_10: 1.41e+00
λ_10 = 1.24e+04+0.00e+00j
Relative error: 9.27e-05
Diff between λ_10 and λ_11: 1.41e+00
λ_11 = 1.24e+04+0.00e+00j
Relative error: 9.07e-05
Diff betwe

# Give guage's orth_eigenvectors to hdf5 files. (pass, don't run this)

In [5]:
params

array([      8,       8,       4,       8, 1048576,       1,       1,
             1,       1,       0,       0,       1,       0,   10000,
             0,       0,       0,       2,       2,       2,       2],
      dtype=int32)

In [39]:
_eigenvectors = io.xxxtzyx2mg_xxxtzyx(input_array=eigenvectors, params=params)
_eigenvectors.shape  # escTtZzYyXx
def orthogonalize(eigenvectors):
    _eigenvectors = eigenvectors.copy()
    size_e, size_s, size_c, size_T, size_t, size_Z, size_z, size_Y, size_y, size_X, size_x = eigenvectors.shape
    print(size_e, size_s, size_c, size_T, size_t,
          size_Z, size_z, size_Y, size_y, size_X, size_x)
    for T in range(size_T):
        for Z in range(size_Z):
            for Y in range(size_Y):
                for X in range(size_X):
                    origin_matrix = eigenvectors[:,
                                                 :, :, T, :, Z, :, Y, :, X, :]
                    _shape = origin_matrix.shape
                    _origin_matrix = origin_matrix.reshape(size_e, -1)
                    condition_number = np.linalg.cond(_origin_matrix.get())
                    print(f"矩阵条件数: {condition_number}")
                    a = _origin_matrix[:, 0]
                    b = _origin_matrix[:, -1]
                    print(cp.dot(a.conj(), b))
                    Q = cp.linalg.qr(_origin_matrix.T)[0]
                    condition_number = np.linalg.cond(Q.get())
                    print(f"矩阵条件数: {condition_number}")
                    a = Q[:, 0]
                    b = Q[:, -1]
                    print(cp.dot(a.conj(), b))
                    _eigenvectors[:, :, :, T, :, Z, :, Y, :, X, :] = Q.T.reshape(
                        _shape)
    return _eigenvectors
orth_eigenvectors = orthogonalize(_eigenvectors)

Input Array Shape: (24, 4, 3, 8, 4, 8, 4)
Dest Shape: (24, 4, 3, 2, 4, 2, 2, 2, 4, 2, 2)
24 4 3 2 4 2 2 2 4 2 2
矩阵条件数: 1.349536418914795
(-0.00022188666-0.00018390932j)
矩阵条件数: 1.0000003576278687
(-6.7520887e-09+6.9849193e-10j)
矩阵条件数: 1.3908592462539673
(0.00010488248-8.790814e-05j)
矩阵条件数: 1.0000004768371582
(4.1909516e-09+1.3271347e-08j)
矩阵条件数: 1.3904129266738892
(0.00024974282-0.0003120522j)
矩阵条件数: 1.0000003576278687
(-3.7252903e-09+0j)
矩阵条件数: 1.361323356628418
(4.209183e-05-0.00011296398j)
矩阵条件数: 1.000000238418579
(1.6763806e-08-1.8626451e-09j)
矩阵条件数: 1.3418352603912354
(0.00011407088+2.1022293e-05j)
矩阵条件数: 1.0000003576278687
(7.450581e-09-2.3283064e-09j)
矩阵条件数: 1.3768424987792969
(-0.00012917638+0.00024368452j)
矩阵条件数: 1.000000238418579
(1.1175871e-08+2.7939677e-09j)
矩阵条件数: 1.3962405920028687
(0.00037449086-0.00015803156j)
矩阵条件数: 1.0000004768371582
(9.313226e-10-4.656613e-09j)
矩阵条件数: 1.3407230377197266
(-0.00025622218-9.2286835e-05j)
矩阵条件数: 1.0000003576278687
(1.8626451e-09-1.8626451

# MultiGrid - give grids.

In [7]:
testvectors = orth_eigenvectors
_src = io.xxxtzyx2mg_xxxtzyx(io.fermion2sctzyx(cp.ones(n, dtype=A.dtype), params), params)

Input Array Shape: (4, 3, 8, 4, 8, 4)
Dest Shape: (4, 3, 2, 4, 2, 2, 2, 4, 2, 2)


In [8]:
_src.shape

(4, 3, 2, 4, 2, 2, 2, 4, 2, 2)

<!-- # MultiGrid - R*vector.
![](./image0-dev40.png) -->

In [9]:
r_src = _src


def r_vec(src):
    return contract("escTtZzYyXx,scTtZzYyXx->eTZYX", testvectors, src)


r_dest = r_vec(r_src)

In [10]:
r_dest.shape

(24, 2, 2, 2, 2)

<!-- # MultiGrid - P*vector.
![](./image1-dev40.png) -->


In [11]:
p_src = r_dest


def p_vec(src):
    return contract("escTtZzYyXx,eTZYX->scTtZzYyXx", cp.conj(testvectors), src)


p_dest = p_vec(p_src)

In [12]:
p_dest.shape

(4, 3, 2, 4, 2, 2, 2, 4, 2, 2)

<!-- # MultiGrid - verify above.
![](./image2-dev40.png) -->

In [13]:
print(cp.linalg.norm(r_src))
print(cp.linalg.norm(p_dest))

110.85125
110.79778


In [14]:
print(cp.linalg.norm(r_src-p_dest)/cp.linalg.norm(r_src))

0.03105482


In [15]:
print(cp.linalg.norm(r_src-p_vec(r_vec(r_src)))/cp.linalg.norm(r_src))

0.03105482


In [16]:
r_src.flatten()[:50]

array([1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
       1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
       1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
       1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
       1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
       1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
       1.+0.j, 1.+0.j], dtype=complex64)

In [17]:
p_dest.flatten()[:50]

array([1.0433708 -1.7586246e-02j, 0.97967035-3.3752520e-02j,
       0.9888903 -2.5632510e-02j, 1.0096966 +1.5668022e-02j,
       0.95146966+4.3760508e-02j, 1.0257877 +6.0996339e-02j,
       0.97537094+6.6072708e-03j, 0.9930297 -6.1011668e-03j,
       1.0111079 -2.7442584e-06j, 0.9699653 -1.2769032e-02j,
       0.9841259 -2.8358703e-04j, 1.0153213 -6.6112755e-03j,
       1.0111208 +5.5497268e-04j, 0.96993566+5.8844220e-05j,
       1.0163035 +1.6793836e-02j, 1.0049931 -1.3325091e-02j,
       1.0077938 +2.4479732e-02j, 0.9943854 +6.2106363e-03j,
       1.0005748 +1.6628278e-03j, 1.0221921 -2.6473200e-02j,
       0.99971396+3.1832088e-02j, 0.98275745+8.7917205e-03j,
       1.0093181 +2.4130852e-03j, 0.9991631 -1.2370172e-02j,
       0.97734064+4.2580470e-02j, 1.0049398 +4.6745227e-03j,
       0.97411805-3.1948414e-02j, 0.9616359 -1.9116245e-02j,
       0.99243784-2.4701818e-03j, 0.9890905 -4.0823076e-02j,
       1.00584   -2.8207920e-02j, 1.0438857 +3.1900711e-02j,
       0.99732   +1.8660

In [18]:
cp.linalg.norm(r_src-p_dest)/cp.linalg.norm(r_src)

array(0.03105482, dtype=float32)

In [19]:
p_vec(r_vec(p_vec(r_vec(p_vec(r_vec(p_vec(r_vec(r_src)))))))).flatten()[:50]

array([1.0433708 -1.75862592e-02j, 0.97967035-3.37524600e-02j,
       0.98889107-2.56325137e-02j, 1.009697  +1.56680923e-02j,
       0.95146966+4.37605567e-02j, 1.025788  +6.09962605e-02j,
       0.9753714 +6.60718558e-03j, 0.9930302 -6.10115612e-03j,
       1.0111077 -2.90037133e-06j, 0.9699653 -1.27690239e-02j,
       0.98412657-2.83725443e-04j, 1.0153216 -6.61116466e-03j,
       1.0111209 +5.54840895e-04j, 0.9699358 +5.89014962e-05j,
       1.0163037 +1.67939495e-02j, 1.0049932 -1.33249378e-02j,
       1.0077934 +2.44796108e-02j, 0.9943851 +6.21065265e-03j,
       1.0005744 +1.66293420e-03j, 1.022192  -2.64731385e-02j,
       0.99971336+3.18322033e-02j, 0.9827569 +8.79192539e-03j,
       1.009318  +2.41326075e-03j, 0.99916273-1.23702977e-02j,
       0.9773403 +4.25806195e-02j, 1.0049394 +4.67439881e-03j,
       0.9741177 -3.19483429e-02j, 0.9616357 -1.91161744e-02j,
       0.9924372 -2.47022370e-03j, 0.9890901 -4.08227928e-02j,
       1.0058392 -2.82079056e-02j, 1.0438855 +3.1900510

In [20]:
cp.linalg.norm(r_src-p_vec(r_vec(p_vec(r_vec(p_vec(r_vec(p_vec(r_vec(r_src)))))))))/cp.linalg.norm(r_src) #???

array(0.03105482, dtype=float32)

In [21]:
# _mat = contract("escTtZzYyXx,escTtZzYyXx->scTtZzYyXx",
#                 testvectors, cp.conj(testvectors)).flatten()
# print(cp.linalg.norm(_mat))
# print(_mat[:100])

# MultiGrid - R*matvec\*P.

In [22]:
def _r_matvec_p(src, matvec):
    return r_vec(matvec(p_vec(io.xxx2eTZYX(src, params))))


def r_matvec_p(src, matvec):
    return io.array2xxx(_r_matvec_p(src, matvec))

# MultiGrid - verify above.

In [23]:
D_r_src = matvec(r_src)

In [24]:
D_r_src.flatten()[:50]

array([13573.798+26.274761j , 13490.745+35.774696j ,
       13499.581+29.655415j , 13536.367-24.394896j ,
       13454.672-46.29932j  , 13556.42 -75.79355j  ,
       13481.258 -5.8800697j, 13513.289 +1.7740173j,
       13524.541 +8.711571j , 13482.487+27.535423j ,
       13508.272 -5.088859j , 13535.1   +2.4020128j,
       13530.387 -1.0551767j, 13480.697 +4.4766364j,
       13540.52 -20.425655j , 13527.419+13.542053j ,
       13530.951-24.043997j , 13522.678 -2.569065j ,
       13523.892 -4.2136564j, 13538.913+33.980064j ,
       13516.804-33.560707j , 13497.535-16.867218j ,
       13527.663 -1.1439672j, 13518.439+18.185316j ,
       13481.743-50.052914j , 13519.732 -4.6215267j,
       13496.091+42.926285j , 13471.3  +16.177689j ,
       13508.061 +8.456775j , 13507.51 +46.11792j  ,
       13521.595+35.034893j , 13571.943-33.601833j ,
       13514.355 -2.2058554j, 13480.999+18.844683j ,
       13533.926+14.511944j , 13512.592 -2.1014147j,
       13498.373+27.22504j  , 13522.993+77.104

In [25]:
p_r_D_p_r_dest=p_vec(_r_matvec_p(r_dest,matvec=matvec))

Input Array Shape: (24, 2, 2, 2, 2)
Dest Shape: (24, 2, 2, 2, 2)


In [26]:
p_r_D_p_r_dest.flatten()[:50]

array([14104.019 -228.40837j  , 13239.675 -450.51248j  ,
       13365.434 -346.38544j  , 13649.685 +203.61621j  ,
       12853.815 +600.51917j  , 13861.721 +821.9647j   ,
       13180.716  +88.59714j  , 13427.011  -88.44182j  ,
       13664.83   +12.109514j , 13100.497 -172.09491j  ,
       13297.031   -3.5118256j, 13718.34   -94.342285j ,
       13657.853   +0.9214573j, 13103.341   -0.98703j  ,
       13742.2295+219.3742j   , 13590.04  -179.23267j  ,
       13621.753 +337.13132j  , 13437.949  +81.07031j  ,
       13515.705  +24.612406j , 13810.512 -357.431j    ,
       13508.315 +423.10147j  , 13288.61  +127.403175j ,
       13632.723  +34.894432j , 13506.463 -166.59174j  ,
       13209.463 +573.57935j  , 13572.172  +68.71375j  ,
       13163.147 -424.31647j  , 12997.682 -253.4448j   ,
       13414.531  -28.144455j , 13372.773 -552.2527j   ,
       13598.688 -388.26367j  , 14100.762 +434.15643j  ,
       13480.207  +22.90073j  , 13052.751 -182.0839j   ,
       13662.841 -171.15176j  ,

In [27]:
cp.linalg.norm(D_r_src-p_r_D_p_r_dest)/cp.linalg.norm(D_r_src)

array(0.03114777, dtype=float32)

In [28]:
r_dest.flatten()[:50]

array([-2.75381947e+01-2.9804075e+00j, -2.75446281e+01-2.9098406e+00j,
       -2.75457573e+01-2.9164839e+00j, -2.75431499e+01-2.9358132e+00j,
       -2.75408669e+01-2.9625549e+00j, -2.75399323e+01-2.9646726e+00j,
       -2.75419693e+01-2.9453528e+00j, -2.75441933e+01-2.9244659e+00j,
       -2.75383987e+01-2.9768014e+00j, -2.75419769e+01-2.9387305e+00j,
       -2.75430069e+01-2.9342146e+00j, -2.75441360e+01-2.9267416e+00j,
       -2.75448112e+01-2.9244461e+00j, -2.75463829e+01-2.9036703e+00j,
       -2.75451584e+01-2.9228601e+00j, -2.75434151e+01-2.9302859e+00j,
        1.59144402e-05-4.3415129e-02j, -1.70517340e-02-3.2780379e-02j,
        1.05371475e-02+1.1419803e-03j, -1.29313767e-02+3.2871217e-03j,
       -5.88530302e-03+2.7386248e-03j, -2.22008228e-02-4.1321963e-03j,
       -2.77023315e-02-2.1144271e-02j, -1.46165788e-02+1.9183993e-02j,
       -1.09072924e-02+9.5222890e-04j,  8.63033161e-03-9.7517371e-03j,
       -1.02344751e-02+2.2263825e-02j, -5.81735373e-03-1.1231750e-02j,
      

In [29]:
p_vec(r_dest).flatten()[:50]

array([1.0433708 -1.7586246e-02j, 0.97967035-3.3752520e-02j,
       0.9888903 -2.5632510e-02j, 1.0096966 +1.5668022e-02j,
       0.95146966+4.3760508e-02j, 1.0257877 +6.0996339e-02j,
       0.97537094+6.6072708e-03j, 0.9930297 -6.1011668e-03j,
       1.0111079 -2.7442584e-06j, 0.9699653 -1.2769032e-02j,
       0.9841259 -2.8358703e-04j, 1.0153213 -6.6112755e-03j,
       1.0111208 +5.5497268e-04j, 0.96993566+5.8844220e-05j,
       1.0163035 +1.6793836e-02j, 1.0049931 -1.3325091e-02j,
       1.0077938 +2.4479732e-02j, 0.9943854 +6.2106363e-03j,
       1.0005748 +1.6628278e-03j, 1.0221921 -2.6473200e-02j,
       0.99971396+3.1832088e-02j, 0.98275745+8.7917205e-03j,
       1.0093181 +2.4130852e-03j, 0.9991631 -1.2370172e-02j,
       0.97734064+4.2580470e-02j, 1.0049398 +4.6745227e-03j,
       0.97411805-3.1948414e-02j, 0.9616359 -1.9116245e-02j,
       0.99243784-2.4701818e-03j, 0.9890905 -4.0823076e-02j,
       1.00584   -2.8207920e-02j, 1.0438857 +3.1900711e-02j,
       0.99732   +1.8660

In [30]:
r_dest.shape

(24, 2, 2, 2, 2)

# MultiGrid - CG (BUG!!!)

In [31]:
# b = r_src.flatten().copy()
# print(b)
# x = cg.slover(b=r_src.flatten(), matvec=matvec,
#               tol=1e-5, max_iter=1000000).run()
# print(b)


In [32]:

# Ax = matvec(x)
# print(cp.linalg.norm(b-Ax)/cp.linalg.norm(b))
# b = r_src.flatten()
# _x = bistabcg.slover(b=b, matvec=matvec,
#                      tol=1e-5, max_iter=1000000).run()
# Ax = matvec(_x)
# print(cp.linalg.norm(b-Ax)/cp.linalg.norm(b))
# #########
# print(cp.linalg.norm(x-_x)/cp.linalg.norm(x))

In [33]:
b = r_src.flatten()
b

array([1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
      dtype=complex64)

In [34]:
# mg_b=r_dest.flatten()
# mg_x = cg.slover(b=mg_b, matvec=functools.partial(r_matvec_p, matvec=matvec),
#                       tol=1e-5, max_iter=1000000).run()
# mg_Ax=r_matvec_p(mg_x,matvec=matvec)
# print(cp.linalg.norm(mg_b-mg_Ax)/cp.linalg.norm(mg_b))
# _mg_x = bistabcg.slover(b=mg_b, matvec=functools.partial(r_matvec_p, matvec=matvec),
#                       tol=1e-5, max_iter=1000000).run()
# mg_Ax=r_matvec_p(_mg_x,matvec=matvec)
# print(cp.linalg.norm(mg_b-mg_Ax)/cp.linalg.norm(mg_b))
# #########
# print(cp.linalg.norm(mg_x-_mg_x)/cp.linalg.norm(mg_x))
# print(cp.linalg.norm(mg_b))

In [35]:
r_dest

array([[[[[-2.75381947e+01-2.98040748e+00j,
           -2.75446281e+01-2.90984058e+00j],
          [-2.75457573e+01-2.91648388e+00j,
           -2.75431499e+01-2.93581319e+00j]],

         [[-2.75408669e+01-2.96255493e+00j,
           -2.75399323e+01-2.96467257e+00j],
          [-2.75419693e+01-2.94535279e+00j,
           -2.75441933e+01-2.92446589e+00j]]],


        [[[-2.75383987e+01-2.97680140e+00j,
           -2.75419769e+01-2.93873048e+00j],
          [-2.75430069e+01-2.93421459e+00j,
           -2.75441360e+01-2.92674160e+00j]],

         [[-2.75448112e+01-2.92444611e+00j,
           -2.75463829e+01-2.90367031e+00j],
          [-2.75451584e+01-2.92286015e+00j,
           -2.75434151e+01-2.93028593e+00j]]]],



       [[[[ 1.59144402e-05-4.34151292e-02j,
           -1.70517340e-02-3.27803791e-02j],
          [ 1.05371475e-02+1.14198029e-03j,
           -1.29313767e-02+3.28712165e-03j]],

         [[-5.88530302e-03+2.73862481e-03j,
           -2.22008228e-02-4.13219631e-03j],
     

# End for CG & BISTABCG. (pass, don't run this)

In [36]:
# cg_solver.end()
# bistabcg_solver.end()

# End for pyqcu. (pass, don't run this)

In [37]:
# qcu.applyEndQcu(set_ptrs, params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_eo_params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_oe_params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_eo_dag_params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_oe_dag_params)